# Crawling [bezdim.org](https://bezdim.org/signali/reports)

### Import Scrapy and install if missing

In [1]:
try:
    import scrapy
except:
    import sys
    !conda install --yes --prefix {sys.prefix} -c conda-forge scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
from time import time
timestamp = time()

In [3]:
import json
import codecs


class JsonWriterPipeline:

    def open_spider(self, spider):
        self.file = codecs.open('reportresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
def parse_description(raw_description):
    striped = [d.strip() for d in raw_description if d.strip()]
    return " ".join(striped)
    
def parse_time_location(when_where_element):
    selectors = when_where_element.xpath('.//span')
        
    date = selectors[0].css('.r_date').extract_first(default='N/A Date')
        
    print(date)
    date = when_where_element.xpath('span.r_date').extract_first(default='N/A Date')
    location = when_where_element.xpath('span.r_location').extract_first(default='N/A Location')
    
    

In [5]:
import logging
from scrapy import Spider
from scrapy.http import Request

class BezDimSpider(Spider):
    name = "bezdim"
    max_page_count = 5
    start_urls = [
        'https://bezdim.org/signali/reports'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
#         'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': 'report-result-{timestamp}.json'.format(timestamp=timestamp)
    }
    
    def start_requests(self):
        for i in range(1, self.max_page_count):
            yield Request('{url}/fetch_reports?page={page_id}'.
                          format(url=self.start_urls[0], page_id=i),
                    callback=self.process_page)
            break
    
    def process_page(self, response):
        for link in response.css('a.r_title'):
            url = link.xpath('@href').extract_first()
            yield Request(url, callback=self.parse)
            break
    
    def parse(self, report):
        title = report.css('h1.report-title::text').extract_first(default='').strip()
        
        raw_desc = report.css('div.report-description-text::text').extract()
        description = parse_description(raw_desc)
        
        parse_time_location(report.css('p.report-when-where'))
        

        yield {
            'title': title,
            'description': description
        }
        
#             description = report.css('div.r_description::text').extract_first().strip()
#             address = report.css('p.bd_location::text').extract_first().strip()
            
            

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BezDimSpider)
process.start()

2019-01-25 20:46:15 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-01-25 20:46:15 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.0 (default, Jun 28 2018, 07:39:16) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.3.1, Platform Darwin-18.2.0-x86_64-i386-64bit
2019-01-25 20:46:15 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'report-result-1548441975.6567721.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


<span class="r_date">09:14 Jan 19 2019 </span>


In [7]:
import pandas as pd

a = pd.read_json('report-result-{timestamp}.json'.format(timestamp=timestamp))

In [8]:
a.description[2]

KeyError: 2